In [116]:
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf
import tensorflow_ranking as tfr

from sklearn.metrics import ndcg_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [117]:
df = pd.read_csv("intern_task.csv")

In [118]:
df

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235253,2,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,...,0.0,0.000000,0.000000,0.471409,0.000000,0.000000,0.001350,0.000002,3.0,1.500000
235254,2,29995,1.0,0.0,1.0,0.0,1.0,0.500000,0.0,0.500000,...,0.0,0.000000,0.000000,0.471409,39.908056,0.000000,0.004850,0.000014,9.0,4.500000
235255,1,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,...,0.0,0.000000,0.000000,0.471409,0.000000,0.000000,0.001064,0.000001,1.0,0.500000
235256,2,29995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [119]:
df.isnull().sum()

rank           0
query_id       0
feature_0      0
feature_1      0
feature_2      0
              ..
feature_139    0
feature_140    0
feature_141    0
feature_142    0
feature_143    0
Length: 146, dtype: int64

In [120]:
df.isnull().sum().sum()

0

In [121]:
df.describe()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
count,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,...,2.352580e+05,2.352580e+05,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000
mean,0.677869,14828.413401,1.911960,0.206233,1.189847,0.550272,1.960082,0.803964,0.097557,0.541174,...,4.763432e+02,1.046698e+04,0.070299,0.740710,11.067947,0.281747,0.027033,0.000269,22.457910,9.560379
std,0.830033,8193.945170,1.237374,0.579089,1.037233,0.790947,1.203534,0.339955,0.266035,0.419973,...,1.954840e+04,2.649584e+06,0.100721,0.352800,15.336861,0.392089,0.033351,0.002088,63.708018,14.347378
min,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8215.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.620522,0.000000,0.000000,0.006703,0.000000,4.000000,2.000000
50%,0.000000,14935.000000,2.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.500000,...,0.000000e+00,0.000000e+00,0.000000,0.926779,9.448900,0.000000,0.017761,0.000008,12.000000,5.500000
75%,1.000000,21580.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1.000000,...,1.000000e+00,4.000000e-01,0.142857,0.999613,16.146733,0.500000,0.034954,0.000109,28.000000,12.000000
max,4.000000,29995.000000,31.000000,18.000000,27.000000,9.000000,31.000000,1.000000,1.000000,1.000000,...,1.731553e+06,8.000000e+08,0.818182,1.000000,2506.799764,1.000000,0.711261,0.250000,15074.000000,1552.000000


# Решим задачу методом poitwise

In [122]:
X_ptw = df.drop(["rank"], axis=1)
y_ptw = df["rank"]

In [123]:
y_ptw

0         0
1         1
2         0
3         1
4         2
         ..
235253    2
235254    2
235255    1
235256    2
235257    1
Name: rank, Length: 235258, dtype: int64

In [124]:
Xtrain_ptw, Xtest_ptw, ytrain_ptw, ytest_ptw = train_test_split(X_ptw, y_ptw, test_size=0.2, random_state=42)

In [125]:
# Создание и обучение линейной регрессии на обучающей выборке
reg = LinearRegression().fit(Xtrain_ptw, ytrain_ptw)

# Оценка качества модели на обучающей выборке
reg.score(Xtrain_ptw, ytrain_ptw)

0.13314609249526

In [126]:
# Предсказание значений целевой переменной на тестовой выборке
test_pred_ptw = reg.predict(Xtest_ptw)

# Создание DataFrame для сравнения истинных и предсказанных значений
y_test_ptw_df = pd.DataFrame({"relevance_score": ytest_ptw, "predicted_ranking": test_pred_ptw})

y_test_ptw_df.head(2)

,relevance_score,predicted_ranking
85742,1,0.223081
175414,1,0.447925


In [127]:
# Сортировка истинных значений по убыванию
true_relevance_ptw = ytest_ptw.sort_values(ascending=False)

# Сортировка предсказанных значений по убыванию
relevance_score_ptw = y_test_ptw_df.sort_values("predicted_ranking", ascending=False)

In [128]:
k_values = [None, 3, 5, 10, 100, 1000, 10000]

# Цикл для расчета nDCG для каждого значения k
for k in k_values:
    if k is None:
        print(f"nDCG score: {ndcg_score(y_true=[true_relevance_ptw.to_numpy()], y_score=[relevance_score_ptw['relevance_score'].to_numpy()])}")
    else:
        print(f"nDCG score @ {k}: {ndcg_score(y_true=[true_relevance_ptw.to_numpy()], y_score=[relevance_score_ptw['relevance_score'].to_numpy()], k=k)}")

nDCG score: 0.9046356554280806
nDCG score @ 3: 0.319164265129683
nDCG score @ 5: 0.319164265129683
nDCG score @ 10: 0.31916426512968304
nDCG score @ 100: 0.31916426512968316
nDCG score @ 1000: 0.3683959163218017
nDCG score @ 10000: 0.5375402862897382


# Подготовим датасет к методам pairwise и listwise(1-ый способ)

In [129]:
X_plw = df.drop("rank", axis=1)
y_plw = df["rank"]

In [130]:
Xtrain_plw, Xtest_plw, ytrain_plw, ytest_plw = train_test_split(X_plw, y_plw, test_size=0.3, random_state=42)

In [131]:
# Получение размеров групп по 'query_id' для обучающей и тестовой выборок
group_train_plw = Xtrain_plw.groupby('query_id').size().tolist()
group_valid_plw = Xtest_plw.groupby('query_id').size().tolist()

In [132]:
len(group_train_plw)

1999

In [133]:
len(group_valid_plw)

1994

In [134]:
X_plw = X_plw.drop("query_id", axis=1)
Xtrain_plw = Xtrain_plw.drop("query_id", axis=1)
Xtest_plw = Xtest_plw.drop("query_id", axis=1)

In [135]:
# Вывод суммарных размеров групп и общего количества строк в обучающей и тестовой выборках
print(sum(group_train_plw), Xtrain_plw.shape[0])
print(sum(group_valid_plw), Xtest_plw.shape[0])

164680 164680
70578 70578


# Решим задачу методом pairwise

In [136]:
# Создание и обучение ранжирующего модели XGBoost с использованием параметров
ranker = xgb.XGBRanker(
    n_estimators=2000,
    learning_rate=0.1,
    objective='rank:pairwise',
    reg_lambda=0.1,
    tree_method='hist',
    device='cuda',
    random_state=42
)

# Обучение модели с использованием групповой информации и вычисления метрики nDCG@5 на валидационной выборке
ranker.fit(
    Xtrain_plw,
    ytrain_plw,
    group=group_train_plw,
    eval_group=[group_valid_plw],
    eval_set=[(Xtest_plw, ytest_plw)],
    eval_metric="ndcg@5",
    verbose=True
)

# Предсказание значений целевой переменной на тестовой выборке
test_pred_prw = ranker.predict(Xtest_plw)

# Создание DataFrame для сравнения истинных и предсказанных значений
y_test_prw_df = pd.DataFrame({"relevance_score": ytest_plw, "predicted_ranking": test_pred_prw})

# Сортировка истинных значений по убыванию
true_relevance_prw = ytest_plw.sort_values(ascending=False)

# Сортировка предсказанных значений по убыванию
relevance_score_prw = y_test_prw_df.sort_values("predicted_ranking", ascending=False)

[0]	validation_0-ndcg@5:0.46376
[1]	validation_0-ndcg@5:0.48904
[2]	validation_0-ndcg@5:0.49523


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-ndcg@5:0.50068
[4]	validation_0-ndcg@5:0.50173
[5]	validation_0-ndcg@5:0.50362
[6]	validation_0-ndcg@5:0.50485
[7]	validation_0-ndcg@5:0.50616
[8]	validation_0-ndcg@5:0.50938
[9]	validation_0-ndcg@5:0.51132
[10]	validation_0-ndcg@5:0.51219
[11]	validation_0-ndcg@5:0.51267
[12]	validation_0-ndcg@5:0.51499
[13]	validation_0-ndcg@5:0.51662
[14]	validation_0-ndcg@5:0.51736
[15]	validation_0-ndcg@5:0.52093
[16]	validation_0-ndcg@5:0.52295
[17]	validation_0-ndcg@5:0.52376
[18]	validation_0-ndcg@5:0.52686
[19]	validation_0-ndcg@5:0.52753
[20]	validation_0-ndcg@5:0.52741
[21]	validation_0-ndcg@5:0.52925
[22]	validation_0-ndcg@5:0.52909
[23]	validation_0-ndcg@5:0.52847
[24]	validation_0-ndcg@5:0.53017
[25]	validation_0-ndcg@5:0.53128
[26]	validation_0-ndcg@5:0.53188
[27]	validation_0-ndcg@5:0.53302
[28]	validation_0-ndcg@5:0.53314
[29]	validation_0-ndcg@5:0.53273
[30]	validation_0-ndcg@5:0.53421
[31]	validation_0-ndcg@5:0.53578
[32]	validation_0-ndcg@5:0.53788
[33]	validation_0

In [137]:
k_values = [None, 3, 5, 10, 100, 1000, 10000]

# Цикл для расчета nDCG для каждого значения k
for k in k_values:
    if k is None:
        print(f"nDCG score: {ndcg_score(y_true=[true_relevance_prw.to_numpy()], y_score=[relevance_score_prw['relevance_score'].to_numpy()])}")
    else:
        print(f"nDCG score @ {k}: {ndcg_score(y_true=[true_relevance_prw.to_numpy()], y_score=[relevance_score_prw['relevance_score'].to_numpy()], k=k)}")

nDCG score: 0.9322750001648934
nDCG score @ 3: 0.5680987202925045
nDCG score @ 5: 0.5680987202925045
nDCG score @ 10: 0.5680987202925046
nDCG score @ 100: 0.5680987202925049
nDCG score @ 1000: 0.5695235143672077
nDCG score @ 10000: 0.6177877315738575


# Решим задачу методом listwise(1-ый способ)

In [138]:
# Создание и обучение ранжирующего модели XGBoost с использованием параметров
ranker = xgb.XGBRanker(
    n_estimators=2000,
    learning_rate=0.1,
    objective='rank:ndcg',
    reg_lambda=0.1,
    tree_method='hist',
    device='cuda',
    random_state=42
)


# Обучение модели с использованием групповой информации и вычисления метрики nDCG@5 на валидационной выборке
ranker.fit(
    Xtrain_plw,
    ytrain_plw,
    group=group_train_plw,
    eval_group=[group_valid_plw],
    eval_set=[(Xtest_plw, ytest_plw)],
    eval_metric="ndcg@5",
    verbose=True
)


# Предсказание значений целевой переменной на тестовой выборке
test_pred_ltw = ranker.predict(Xtest_plw)

# Создание DataFrame для сравнения истинных и предсказанных значений
y_test_ltw_df = pd.DataFrame({"relevance_score": ytest_plw, "predicted_ranking": test_pred_ltw})

# Сортировка истинных значений по убыванию
true_relevance_ltw = ytest_plw.sort_values(ascending=False)

# Сортировка предсказанных значений по убыванию
relevance_score_ltw = y_test_ltw_df.sort_values("predicted_ranking", ascending=False)

[0]	validation_0-ndcg@5:0.47204
[1]	validation_0-ndcg@5:0.49207
[2]	validation_0-ndcg@5:0.50220


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-ndcg@5:0.50275
[4]	validation_0-ndcg@5:0.50032
[5]	validation_0-ndcg@5:0.50015
[6]	validation_0-ndcg@5:0.50307
[7]	validation_0-ndcg@5:0.50326
[8]	validation_0-ndcg@5:0.50394
[9]	validation_0-ndcg@5:0.50565
[10]	validation_0-ndcg@5:0.50535
[11]	validation_0-ndcg@5:0.50671
[12]	validation_0-ndcg@5:0.50768
[13]	validation_0-ndcg@5:0.50811
[14]	validation_0-ndcg@5:0.51054
[15]	validation_0-ndcg@5:0.51151
[16]	validation_0-ndcg@5:0.51181
[17]	validation_0-ndcg@5:0.51274
[18]	validation_0-ndcg@5:0.51350
[19]	validation_0-ndcg@5:0.51412
[20]	validation_0-ndcg@5:0.51429
[21]	validation_0-ndcg@5:0.51546
[22]	validation_0-ndcg@5:0.51586
[23]	validation_0-ndcg@5:0.51706
[24]	validation_0-ndcg@5:0.51752
[25]	validation_0-ndcg@5:0.51832
[26]	validation_0-ndcg@5:0.51917
[27]	validation_0-ndcg@5:0.51936
[28]	validation_0-ndcg@5:0.52047
[29]	validation_0-ndcg@5:0.52184
[30]	validation_0-ndcg@5:0.52322
[31]	validation_0-ndcg@5:0.52342
[32]	validation_0-ndcg@5:0.52268
[33]	validation_0

In [139]:
k_values = [None, 3, 5, 10, 100, 1000, 10000]

# Цикл для расчета nDCG для каждого значения k
for k in k_values:
    if k is None:
        print(f"nDCG score: {ndcg_score(y_true=[true_relevance_ltw.to_numpy()], y_score=[relevance_score_ltw['relevance_score'].to_numpy()])}")
    else:
        print(f"nDCG score @ {k}: {ndcg_score(y_true=[true_relevance_ltw.to_numpy()], y_score=[relevance_score_ltw['relevance_score'].to_numpy()], k=k)}")

nDCG score: 0.9354198831279984
nDCG score @ 3: 0.6677330895795247
nDCG score @ 5: 0.6677330895795247
nDCG score @ 10: 0.6677330895795248
nDCG score @ 100: 0.667733089579525
nDCG score @ 1000: 0.6469392625550074
nDCG score @ 10000: 0.6295715314202055


# Подготовим датасет для метода listwise(2-ой способ)

In [140]:
!pip install -q tensorflow-ranking

In [141]:
# Разделение данных
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Функция предобработки данных
def preprocess(data):
    features = {
        "float_features": data.iloc[:, 2:].values.astype(np.float32),
        "label": data['rank'].values.astype(np.float32),
        "query_id": data['query_id'].values.astype(np.int32)
    }
    return features

# Создание датасетов
def create_dataset(df):
    ds = tf.data.Dataset.from_tensor_slices(preprocess(df))
    ds = ds.map(lambda feature_map: {
        "_mask": tf.ones_like(feature_map["label"], dtype=tf.bool),
        **feature_map
    })
    ds = ds.shuffle(buffer_size=1000).padded_batch(batch_size=32)
    ds = ds.map(lambda feature_map: (
        {k: v for k, v in feature_map.items() if k != "label"},
        tf.expand_dims(tf.where(feature_map["_mask"], feature_map["label"], -1.), axis=-1)
    ))
    return ds

train_ds = create_dataset(train_df)
test_ds = create_dataset(test_df)


# Решим задачу методом listwise(2-ой способ)

In [142]:
# Создание модели
inputs = {
    "float_features": tf.keras.Input(shape=(144,), dtype=tf.float32)
}

# Объединение всех входов в один тензор
x = tf.concat(list(inputs.values()), axis=-1)

# Применение BatchNormalization к объединенному тензору
x = tf.keras.layers.BatchNormalization()(x)

for layer_width in [128, 64, 32]:
    x = tf.keras.layers.Dense(units=layer_width)(x)
    x = tf.keras.layers.Activation(activation=tf.nn.relu)(x)
scores = tf.keras.layers.Dense(units=1)(x)

# Обучение модели
model = tf.keras.Model(inputs=inputs, outputs=scores)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tfr.keras.losses.SoftmaxLoss(),
    metrics=[tfr.keras.metrics.NDCGMetric(name="ndcg_5", topn=5)]
)
model.fit(train_ds, epochs=3)


Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['_mask', 'query_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


5882/5882 [==============================] - 70s 12ms/step - loss: 0.0000e+00 - ndcg_5: 0.4833
Epoch 2/3
5882/5882 [==============================] - 41s 7ms/step - loss: 0.0000e+00 - ndcg_5: 0.4833
Epoch 3/3
5882/5882 [==============================] - 42s 7ms/step - loss: 0.0000e+00 - ndcg_5: 0.4833


In [144]:
# Оценка модели
results = model.evaluate(test_ds)
print(f"nDCG score @ 5: {results[1]}")

1471/1471 [==============================] - 9s 6ms/step - loss: 0.0000e+00 - ndcg_5: 0.4843
nDCG score @ 5: 0.484251469373703
